<a href="https://colab.research.google.com/github/XueweiZhao/XueweiZhao.github.io/blob/master/FHIRViews_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparations

## Install the latest google-fhir-views and import packages

In [10]:
!pip install --upgrade -q google-fhir-views[r4,bigquery]==0.10.0

import datetime
from google.colab import auth
from google.cloud import bigquery
from google.fhir.views import bigquery_runner, r4

## Authentication and defining BigQuery client/runner and views

In [31]:
auth.authenticate_user()
print('Authenticated')

client = bigquery.Client(project='dqi-playground-teams')
runner = bigquery_runner.BigQueryRunner(
    client,
    fhir_dataset='synthea_fhirstore')

views = r4.base_r4()

# Add some valuesets to the views
VALUESET_URI = 'urn:example:value_set:ldl'
LDL_TEST = r4.value_set(VALUESET_URI).with_codes(
    'http://loinc.org', ['18262-6', '18261-8', '12773-8']).build()
views._context.add_local_value_set(LDL_TEST)

Authenticated


# [New Feature!] views.from_view_definition(a_json_format_view_defination)

## Example of simple Patient view

In [30]:
simple_pats_config = {
    'resource': 'Patient',
    'select': [
        {
            'alias': 'id',
            'path': 'id',
        },
        {
            'alias': 'gender',
            'path': 'gender',
        },
        {
            'alias': 'birthDate',
            'path': 'birthDate',
        },
        {
            'alias': 'street',
            'path': 'address.where(address.period.empty()).first().line.first()',
        },
        {
            'alias': 'city',
            'path': 'address.where(address.period.empty()).first().city',
        },
        {
            'alias': 'state',
            'path': 'address.where(address.period.empty()).first().state',
        },
        {
            'alias': 'zip',
            'path': 'address.where(address.period.empty()).first().postalCode',
        },
    ],
    'where': [
        {'path': 'birthDate < @1990-01-01'},
    ],
}

simple_pats_view_from_definition = views.from_view_definition(simple_pats_config)

runner.to_dataframe(simple_pats_view_from_definition, limit = 5)

,id,gender,birthDate,street,city,state,zip
0,016628e3-a50d-b351-aed1-81bef2a02fd1,female,1980-06-29,1009 Buckridge Crossing,Woburn,MA,01801
1,b1fe69f1-368d-b0e6-275f-4d977baaf06e,male,1989-10-07,491 Rempel Promenade,Methuen,MA,None
2,d05e8bbf-9511-7fe9-dd85-dff81b266ea8,female,1984-04-05,360 McLaughlin Byway Apt 13,Marlborough,MA,01752
3,8f009f08-335c-d7a4-a1fa-8706c7c29169,female,1971-01-30,1085 Trantow Ville Apt 56,Fall River,MA,02721
4,566a9f50-7385-9f89-9db7-18b03f190e34,female,1961-09-21,128 Monahan Trafficway Unit 35,Chicopee,MA,01013


## Example of filtered Observation view

In [32]:
ldl_obs_config = {
    'resource': 'Observation',
    'select': [
        {
            'alias': 'patient',
            'path': "subject.idFor('Patient')",
        },
        {
            'alias': 'value',
            'path': "value.ofType('Quantity').value",
        },
        {
            'alias': 'unit',
            'path': "value.ofType('Quantity').unit",
        },
        {
            'alias': 'test',
            'path': 'code.coding.display.first()',
        },
        {
            'alias': 'effectiveTime',
            'path': "effective.ofType('datetime')",
        },
    ],
    'where': [
        {'path': f"code.memberOf('{VALUESET_URI}')"},
    ],
}

ldl_obs_view_from_definition = views.from_view_definition(ldl_obs_config)

runner.to_dataframe(ldl_obs_view_from_definition, limit=5)

,patient,value,unit,test,effectiveTime
0,566a9f50-7385-9f89-9db7-18b03f190e34,121.74,mg/dL,Low Density Lipoprotein Cholesterol,2016-06-02 21:12:13+00:00
1,566a9f50-7385-9f89-9db7-18b03f190e34,79.50,mg/dL,Low Density Lipoprotein Cholesterol,2019-06-20 21:12:13+00:00
2,f12bbe04-9958-c1d0-eaee-11a084ea877f,126.86,mg/dL,Low Density Lipoprotein Cholesterol,2018-11-27 10:35:41+00:00
3,566a9f50-7385-9f89-9db7-18b03f190e34,11.21,mg/dL,Low Density Lipoprotein Cholesterol,2013-05-16 21:12:13+00:00
4,b1fe69f1-368d-b0e6-275f-4d977baaf06e,65.07,mg/dL,Low Density Lipoprotein Cholesterol,2020-02-08 16:13:33+00:00


# [New Feature!] views.select([a_list_of_fhir_paths_with_alias])

## Example of simple Patient view (equivalent to above)

In [33]:
pats = views.view_of('Patient')

simple_pats_view_from_list = pats.select([
    pats.id.alias('id'),
    pats.gender.alias('gender'),
    pats.birthDate.alias('birthdate'),
    pats.address.where(pats.address.period.empty()).first().line.first().alias('street'),
    pats.address.where(pats.address.period.empty()).first().city.alias('city'),
    pats.address.where(pats.address.period.empty()).first().state.alias('state'),
    pats.address.where(pats.address.period.empty()).first().postalCode.alias('zip')
    ]).where(
       pats.birthDate < datetime.date(1990,1,1))

runner.to_dataframe(simple_pats_view_from_list, limit = 5)

,id,gender,birthdate,street,city,state,zip
0,016628e3-a50d-b351-aed1-81bef2a02fd1,female,1980-06-29,1009 Buckridge Crossing,Woburn,MA,01801
1,b1fe69f1-368d-b0e6-275f-4d977baaf06e,male,1989-10-07,491 Rempel Promenade,Methuen,MA,None
2,d05e8bbf-9511-7fe9-dd85-dff81b266ea8,female,1984-04-05,360 McLaughlin Byway Apt 13,Marlborough,MA,01752
3,8f009f08-335c-d7a4-a1fa-8706c7c29169,female,1971-01-30,1085 Trantow Ville Apt 56,Fall River,MA,02721
4,566a9f50-7385-9f89-9db7-18b03f190e34,female,1961-09-21,128 Monahan Trafficway Unit 35,Chicopee,MA,01013


## Example of filtered Observation view (equivalent to above)

In [34]:
obs = views.view_of('Observation')

ldl_obs_view_from_list = obs.select([
    obs.subject.idFor('Patient').alias('patient'),
    obs.valueQuantity.value.alias('value'),
    obs.valueQuantity.unit.alias('unit'),
    obs.code.coding.display.first().alias('test'),
    obs.effectiveDateTime.alias('effectiveTime')
    ]).where(obs.code.memberOf(LDL_TEST))

runner.to_dataframe(ldl_obs_view_from_list, limit = 5)

,patient,value,unit,test,effectiveTime
0,566a9f50-7385-9f89-9db7-18b03f190e34,121.74,mg/dL,Low Density Lipoprotein Cholesterol,2016-06-02 21:12:13+00:00
1,566a9f50-7385-9f89-9db7-18b03f190e34,79.50,mg/dL,Low Density Lipoprotein Cholesterol,2019-06-20 21:12:13+00:00
2,f12bbe04-9958-c1d0-eaee-11a084ea877f,126.86,mg/dL,Low Density Lipoprotein Cholesterol,2018-11-27 10:35:41+00:00
3,566a9f50-7385-9f89-9db7-18b03f190e34,11.21,mg/dL,Low Density Lipoprotein Cholesterol,2013-05-16 21:12:13+00:00
4,b1fe69f1-368d-b0e6-275f-4d977baaf06e,65.07,mg/dL,Low Density Lipoprotein Cholesterol,2020-02-08 16:13:33+00:00


# The old way of views.select({column_name: fhir_path}) is still supported

## Example of simple Patient view (equivalent to above)

In [17]:
simple_pats_view_from_map = pats.select({
    'id': pats.id,
    'gender': pats.gender,
    'birthdate': pats.birthDate,
    'street': pats.address.where(pats.address.period.empty()).first().line.first(),
    'city': pats.address.where(pats.address.period.empty()).first().city,
    'state': pats.address.where(pats.address.period.empty()).first().state,
    'zip': pats.address.where(pats.address.period.empty()).first().postalCode,
    }).where(
       pats.birthDate < datetime.date(1990,1,1))

runner.to_dataframe(simple_pats_view_from_map, limit = 5)

,id,gender,birthdate,street,city,state,zip
0,016628e3-a50d-b351-aed1-81bef2a02fd1,female,1980-06-29,1009 Buckridge Crossing,Woburn,MA,01801
1,b1fe69f1-368d-b0e6-275f-4d977baaf06e,male,1989-10-07,491 Rempel Promenade,Methuen,MA,None
2,d05e8bbf-9511-7fe9-dd85-dff81b266ea8,female,1984-04-05,360 McLaughlin Byway Apt 13,Marlborough,MA,01752
3,8f009f08-335c-d7a4-a1fa-8706c7c29169,female,1971-01-30,1085 Trantow Ville Apt 56,Fall River,MA,02721
4,566a9f50-7385-9f89-9db7-18b03f190e34,female,1961-09-21,128 Monahan Trafficway Unit 35,Chicopee,MA,01013


## Example of filtered Observation view (equivalent to above)

In [35]:
ldl_obs_view_from_map = obs.select({
    'patient': obs.subject.idFor('Patient'),
    'value': obs.valueQuantity.value,
    'unit': obs.valueQuantity.unit,
    'test': obs.code.coding.display.first(),
    'effectiveTime': obs.effectiveDateTime,
    }).where(obs.code.memberOf(LDL_TEST))

runner.to_dataframe(ldl_obs_view_from_map, limit = 5)

,patient,value,unit,test,effectiveTime
0,566a9f50-7385-9f89-9db7-18b03f190e34,121.74,mg/dL,Low Density Lipoprotein Cholesterol,2016-06-02 21:12:13+00:00
1,566a9f50-7385-9f89-9db7-18b03f190e34,79.50,mg/dL,Low Density Lipoprotein Cholesterol,2019-06-20 21:12:13+00:00
2,f12bbe04-9958-c1d0-eaee-11a084ea877f,126.86,mg/dL,Low Density Lipoprotein Cholesterol,2018-11-27 10:35:41+00:00
3,566a9f50-7385-9f89-9db7-18b03f190e34,11.21,mg/dL,Low Density Lipoprotein Cholesterol,2013-05-16 21:12:13+00:00
4,b1fe69f1-368d-b0e6-275f-4d977baaf06e,65.07,mg/dL,Low Density Lipoprotein Cholesterol,2020-02-08 16:13:33+00:00
